### Notebook for visualization potential ground truth dataset

want to mark ethograms and vids that could be used for ground truth

In [1]:
import pandas as pd
from ipywidgets import HBox, VBox, Select, Button, Layout
from fastplotlib import ImageWidget, Plot
from mesmerize_core.arrays import LazyVideo
from pathlib import Path
import numpy as np
from fastplotlib.graphics.selectors import LinearSelector
import pickle

2023-06-13 16:23:36.999887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 16:23:37.447053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ETHOGRAM_COLORS = {
    "lift": "b",
    "handopen": "green",
    "grab": "r",
    "sup": "cyan",
    "atmouth": "magenta",
    "chew": "yellow"
}

In [3]:
df = pd.read_hdf("/data/caitlin/exactly1_hand_labels.hdf")

In [4]:
df

,mat_path,hand_labels,jaaba_labels,merged_ethogram
0,M232_20170306,"{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170306_v024': [[0, 0, 0, 0, 0, 0, 0, ..."
1,M232_20170307,"{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170307_v064': [[0, 0, 0, 0, 0, 0, 0, ..."
2,M232_20170308,"{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170308_v028': [[0, 0, 0, 0, 0, 0, 0, ..."
3,M232_20170314,"{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ...","{'M232_20170314_v029': [[0, 0, 0, 0, 0, 0, 0, ..."
4,M234_20170328,"{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ...","{'M234_20170328_v002': [[0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
445,M240_20170724FinalChecked,"{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170724_v014': [[0, 0, 0, 0, 0, 0, 0, ..."
446,M240_20170725FinalChecked,"{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170725_v031': [[0, 0, 0, 0, 0, 0, 0, ..."
447,M240_20170727FinalChecked,"{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170727_v007': [[0, 0, 0, 0, 0, 0, 0, ..."
448,M240_20170802FinalChecked,"{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ...","{'M240_20170802_v017': [[0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
ethograms = dict()

In [6]:
for row in df.iterrows():
    keys = row[1]['merged_ethogram'].keys()
    for k in keys:
        ethograms[k] = row[1]['merged_ethogram'][k]

In [7]:
data_path = Path('/data/caitlin/potential_ground_truth/')
options = [k.stem for k in sorted(data_path.glob('*.avi'))]

In [8]:
len(options)

458

In [9]:
def trial_change(obj):
    selected_vid = data_path.joinpath(trial_selector.value).with_suffix('.avi')
    iw.set_data([LazyVideo(selected_vid)], reset_vmin_vmax=True)
    plot.clear()
    make_plot(plot)
    plot.auto_scale()

In [10]:
# columns = ["trial", "mark"]
# df2 = pd.DataFrame(columns=columns)
# df2.to_hdf('/data/caitlin/mark_trials.hdf', key='df')
df2 = pd.read_hdf('/data/caitlin/mark_trials.hdf')
df2

,trial,mark


In [11]:
def mark_good_trial(obj):
    try:
        df2[trial_selector.value]
        df2[df2[trial_selector.value]]["trial_status"] = "good"
    except KeyError:
        df2.loc[len(df2.index)] = [trial_selector.value, "good"]
    df2.to_hdf('/data/caitlin/mark_trials.hdf', key='df')
def mark_bad_trial(obj):
    try:
        df2[trial_selector.value]
        df2[df2[trial_selector.value]]["trial_status"] = "bad"
    except KeyError:
        df2.loc[len(df2.index)] = [trial_selector.value, "bad"]
    df2.to_hdf('/data/caitlin/mark_trials.hdf', key='df')
def mark_clean_trial(obj):
    try:
        df2[trial_selector.value]
        df2[df2[trial_selector.value]]["trial_status"] = "clean"
    except KeyError:
        df2.loc[len(df2.index)] = [trial_selector.value, "clean"]
    df2.to_hdf('/data/caitlin/mark_trials.hdf', key='df')

In [12]:
trial_selector = Select(options=options)
trial_selector.observe(trial_change, "value")
good_trial_button = Button(value=False, disabled=False, icon='check',
                                       layout=Layout(width='auto'), tooltip='select trial as train')
bad_trial_button = Button(value=False, disabled=False, icon='times',
                                       layout=Layout(width='auto'), tooltip='throwaway trial')
clean_trial_button = Button(value=False, disabled=False, icon='wrench',
                                       layout=Layout(width='auto'), tooltip='clean trial')
good_trial_button.on_click(mark_good_trial)
bad_trial_button.on_click(mark_bad_trial)
clean_trial_button.on_click(mark_clean_trial)

In [13]:
iw = ImageWidget(data=[LazyVideo(data_path.joinpath(trial_selector.value).with_suffix('.avi'))])

RFBOutputContext()

/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:87: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
/home/clewis7/repos/mesmerize-core/mesmerize_core/arrays/_video.py:92: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


In [14]:
def ethogram_event_handler(ev):
    ix = ev.pick_info["selected_index"]
    iw.sliders["t"].value = ix

In [15]:
plot = Plot(size=(500, 100))
def make_plot(plot):
        
    ethogram_array = ethograms[trial_selector.value]
    y_bottom = 0
    for i, b in enumerate(ETHOGRAM_COLORS.keys()):
        xs = np.arange(ethogram_array.shape[1], dtype=np.float32)
        ys = np.zeros(xs.size, dtype=np.float32)

        lg = plot.add_line(
            data=np.column_stack([xs, ys]),
            thickness=10,
            name=b
        )

        lg.colors = 0
        lg.colors[ethogram_array[i] == 1] = ETHOGRAM_COLORS[b]

        y_pos = (i * -10) - 1
        lg.position_y = y_pos

    ethogram_selector = LinearSelector(
                selection=0,
                limits=(0, ethogram_array.shape[1]),
                axis="x",
                parent=lg,
                end_points=(y_bottom, y_pos),
            )

    plot.add_graphic(ethogram_selector)
    ethogram_selector.selection.add_event_handler(ethogram_event_handler)

RFBOutputContext()

In [16]:
make_plot(plot)
VBox([
    HBox([iw.show(), 
          VBox([
              trial_selector,
              HBox([
                  good_trial_button,
                  clean_trial_button,
                  bad_trial_button
              ])
            ])
         ]),
    plot.show()
])

In [17]:
df2

,trial,mark
